In [ ]:
conda install -c districtdatalabs yellowbrick

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.6.14
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /srv/conda/envs/notebook

  added / updated specs:
    - yellowbrick


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    atomicwrites-1.3.0         |             py_0           9 KB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    certifi-2019.9.11          |           py37_0         147 KB  conda-forge
    importlib_metadata-0.23    |           py37_0          39 KB  conda-forge
    more-itertools-7.2.0       |             py_0          48 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    pluggy-0.13.0              |           py37_0          28 K

## Uffi Nadzima
### Final Project BDA DTS UNS 2019

## Importing Required Libraries

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy import stats
from sklearn.linear_model import LinearRegression
from sklearn import datasets, linear_model, metrics 
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

## Exploring Data

In [ ]:
data=pd.read_csv("heart.csv")
data.head()

Terdapat 10 variable:

1.  sbp         : tekanan darah sistolik
2.  tobacco     : Konsumsi tembakau kumulatif, dalam kg
3.  ldl         : kolesterol lipoprotein densitas rendah
4.  adiposity   : konsentrasi jaringan adiposa
5.  famhist     : Sejarah keluarga penyakit jantung (1 = Sekarang, 0 = Tidak Ada)
6.  typea       : Skor pada tes yang dirancang untuk mengukur perilaku tipe-A
7.  obesity     : Obesitas
8.  alcohol     : Konsumsi alkohol saat ini
9.  age         : usia subjek
10. chd         : Penyakit jantung koroner pada awal; 1 = Ya 0 = Tidak

Setiap baris berikut berisi informasi dari satu pasien. Ada total 462 sampel.

In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
data.count()

In [ ]:
np.sum(data.isnull())

In [ ]:
data.corr()

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))
sns.heatmap(data.corr(), center=0, annot=True, cmap="YlGnBu")

## Scatter Plot

In [ ]:
plt.scatter(data['adiposity'],data['obesity'])
plt.xlabel('Konsentrasi Jaringan Adiposa')
plt.ylabel('Obesitas')
plt.show()

In [ ]:
data2=data.filter(items=['adiposity', 'obesity'])
data2

## Removing Outlier

In [ ]:
meanadiposity = data['adiposity'].mean()
stdadiposity = data['adiposity'].std()
meanobesity = data['obesity'].mean()
stdobesity = data['obesity'].std()

databersih=data[(np.abs((data['adiposity']-meanadiposity)/stdadiposity)<=2)
               & (np.abs((data['obesity']-meanobesity)/stdobesity)<=2)]

databersih

In [ ]:
plt.scatter(databersih['adiposity'],databersih['obesity'])
plt.show()

## Normalisasi Data

In [ ]:
normalx =databersih['adiposity']/max(databersih['adiposity'])
normaly =databersih['obesity']/max(databersih['obesity'])

## Regresi Polinomial

In [ ]:
def polinomial(x, b0, b1, b2):
    y = b0+(b1*x)+(b2*(x**2))
    return y
popt, pcov = curve_fit(polinomial, normalx, normaly)
print("POLINOMIAL: b0 = %f, b1 = %f, b2 = %f " % (popt[0], popt[1], popt[2]))
x = databersih['adiposity']/max(databersih['adiposity'])
plt.figure(figsize=(8,5))
y = polinomial(x, *popt)
plt.plot(normalx, normaly, 'ro', label='data')
plt.plot(x,y, linewidth=2.0, label='polinomial')
plt.legend(loc='best')
plt.ylabel('Konsentrasi Jaringan Adiposa')
plt.xlabel('Obesitas')
plt.show()

In [ ]:
X=np.array(normalx).reshape(-1,1)
Y=np.array(normaly).reshape(-1,1)
X_train, X_test, y_train, y_test=train_test_split(X,Y,test_size=0.10,random_state=145)
popt, pcov = curve_fit(polinomial, X_train.ravel(), y_train.ravel())
y_topi = polinomial(X_test, *popt)
print('Mean absolute error: %.4f' % np.mean(np.absolute(y_test-y_topi)))
print('Residual sum of squares (MSE): %.4f' % np.mean((y_test-y_topi)**2))
print('R-sq: %.4f' % r2_score(y_topi, y_test))

## Regresi Sigmoid

In [ ]:
def sigmoid(x, b1, b2):
    y = 1 / (1 + np.exp(-b1*(x-b2)))
    return y
popt, pcov = curve_fit(sigmoid, normalx, normaly)
print(" b1 = %f, b2 = %f" % (popt[0], popt[1]))
x = databersih['adiposity']/max(databersih['adiposity'])
plt.figure(figsize=(8,5))
y = sigmoid(x, *popt)
plt.plot(normalx, normaly, 'ro', label='data')
plt.plot(x,y, linewidth=3.0, label='sigmoid')
plt.legend(loc='best')
plt.ylabel('Konsentrasi Jaringan Adiposa')
plt.xlabel('Obesitas')
plt.show()

In [ ]:
X=np.array(normalx).reshape(-1,1)
Y=np.array(normaly).reshape(-1,1)
X_train, X_test, y_train, y_test=train_test_split(X,Y,test_size=0.20,random_state=123)
popt, pcov = curve_fit(sigmoid, X_train.ravel(), y_train.ravel())
y_topi = sigmoid(X_test, *popt)
print('Mean absolute error: %.4f' % np.mean(np.absolute(y_test-y_topi)))
print('Residual sum of squares (MSE): %.4f' % np.mean((y_test-y_topi)**2))
print('R-sq: %.4f' % r2_score(y_topi, y_test))

## Regresi Eksponensial

In [ ]:
def eksponensial(x, b0, b1):
    y = b0*np.exp(b1*x)
    return y
popt, pcov = curve_fit(eksponensial, normalx, normaly)
print(" b1 = %f, b2 = %f" % (popt[0], popt[1]))
x = databersih['adiposity']/max(databersih['adiposity'])
plt.figure(figsize=(8,5))
y = eksponensial(x, *popt)
plt.plot(normalx, normaly, 'ro', label='data',)
plt.plot(x,y, linewidth=3.0, label='eksponensial')
plt.legend(loc='best')
plt.ylabel('Konsentrasi Jaringan Adiposa')
plt.xlabel('Obesitas')
plt.show()

In [ ]:
Y=np.array(normaly).reshape(-1,1)
X_train, X_test, y_train, y_test=train_test_split(X,Y,test_size=0.20,random_state=123)
popt, pcov = curve_fit(eksponensial, X_train.ravel(), y_train.ravel())
y_topi = eksponensial(X_test, *popt)
print('Mean absolute error: %.4f' % np.mean(np.absolute(y_test-y_topi)))
print('Residual sum of squares (MSE): %.4f' % np.mean((y_test-y_topi)**2))
print('R-sq: %.4f' % r2_score(y_topi, y_test))

## Regresi linear

In [ ]:
def linear(x, b0, b1):
    y = b0+(b1*x)
    return y
popt, pcov = curve_fit(linear, normalx, normaly)
print(" b1 = %f, b2 = %f" % (popt[0], popt[1]))
x = np.linspace(1960, 2015, 55)
x = databersih['adiposity']/max(databersih['adiposity'])
plt.figure(figsize=(8,5))
y = linear(x, *popt)
plt.plot(normalx, normaly, 'ro', label='data')
plt.plot(x,y, linewidth=3.0, label='linear')
plt.legend(loc='best')
plt.ylabel('Konsentrasi Jaringan Adiposa')
plt.xlabel('Obesitas')
plt.show()

In [ ]:
X=np.array(normalx).reshape(-1,1)
Y=np.array(normaly).reshape(-1,1)
X_train, X_test, y_train, y_test=train_test_split(X,Y,test_size=0.20,random_state=123)
popt, pcov = curve_fit(linear, X_train.ravel(), y_train.ravel())
y_topi = linear(X_test, *popt)
print('Mean absolute error: %.4f' % np.mean(np.absolute(y_test-y_topi)))
print('Residual sum of squares (MSE): %.4f' % np.mean((y_test-y_topi)**2))
print('R-sq: %.4f' % r2_score(y_topi, y_test))

## Evaluasi Model

Evaluasi model dilakukan dengan membandingkan nilai *__Mean Absolute Error(MAE)__*, *__Residual sum of squares/Mean Squaare Error(MSE)__*, serta *__R2-score__*.

1. __Regresi Polinomial__   ->  
                                Mean absolute error: 0.0481
                                Residual sum of squares (MSE): 0.0040
                                R-sq: 0.1801
2. __Regresi Sigmoid__      ->  
                                Mean absolute error: 0.0494
                                Residual sum of squares (MSE): 0.0040
                                R-sq: 0.1994
3. __Regresi Eksponensial__ ->  
                                Mean absolute error: 0.0508
                                Residual sum of squares (MSE): 0.0040
                                R-sq: 0.2113
4. __Regresi Linear__       ->  
                                Mean absolute error: 0.0499
                                Residual sum of squares (MSE): 0.0040
                                R-sq: 0.2271

Semakin kecil nilai __MAE__ dan __MSE__ serta semakin tinggi nilai __R2 score__ model regresinya semakin baik.
Dari evaluasi model regresi diatas, diperoleh model yang sesuai adalah model regresi linear. Hal ini dikarenakan pada model regresi linear nilai MAE dan MSE nya kecil dan nilai R2-scorenya lebih besar dibandingkan dengan model yang lain.

## Simple Regression

In [ ]:
data2=databersih.filter(items=['ldl','obesity'])
data2.head()

In [ ]:
x = data2['ldl'].values.reshape(-1,1)
y = data2['obesity'].values.reshape(-1,1)

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test=train_test_split(x,y, test_size=1/3,random_state=0)

In [ ]:
from sklearn.linear_model import LinearRegression
regressor=LinearRegression()
regressor.fit(x_train,y_train)

In [ ]:
print('intercept:',regressor.intercept_)
print('slope:',regressor.coef_)

Dari hasil di atas diperoleh model regresi sederhana yaitu $y=0.35523342x+16.70266666$ artinya untuk setiap satuan __adiposity__ akan meningkatkan $0.35523342$ __obesity__ 

In [ ]:
y_pred = regressor.predict(x_test)

In [ ]:
df = pd.DataFrame({'Actual': y_test.flatten(), 'Predicted': y_pred.flatten()})
df

## Validasi Data

In [ ]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn import metrics

print('Mean absolute error: %.2f' % np.mean(np.absolute(y_pred-y_test)))
print('MSE: %.2f' % mean_squared_error(y_test, y_pred))
from sklearn.metrics import r2_score
print('R2-score: %.2f' % r2_score(y_test, y_pred))

In [ ]:
def mean_absolute_percentage_error(y_test, y_pred): 
    y_test, y_pred = np.array(y_test), np.array(y_pred)
    return np.mean(np.abs((y_test - y_pred) / y_test)) * 100
mean_absolute_percentage_error(y_test, y_pred)

## Clustering

In [ ]:
X=databersih.filter(items=['tobacco','adiposity'])
X.head()

In [ ]:
X=X.values

In [ ]:
from sklearn.cluster import KMeans
from yellowbrick.cluster import KElbowVisualizer

In [ ]:
model = KMeans(random_state=0)
visualizer = KElbowVisualizer(model, k=(2,10), metric='silhouette', timings=False)

In [ ]:
visualizer.fit(X)    
visualizer.poof()

In [ ]:
kmeans = KMeans(n_clusters = 5, init = 'k-means++', random_state = 0)
y_kmeans = kmeans.fit_predict(X)

In [ ]:
plt.scatter(X[y_kmeans == 0, 0], X[y_kmeans == 0, 1], s = 100, c = 'red', label = 'Cluster 1')
plt.scatter(X[y_kmeans == 1, 0], X[y_kmeans == 1, 1], s = 100, c = 'blue', label = 'Cluster 2')
plt.scatter(X[y_kmeans == 2, 0], X[y_kmeans == 2, 1], s = 100, c = 'green', label = 'Cluster 3')
plt.scatter(X[y_kmeans == 3, 0], X[y_kmeans == 3, 1], s = 100, c = 'cyan', label = 'Cluster 4')
plt.scatter(X[y_kmeans == 4, 0], X[y_kmeans == 4, 1], s = 100, c = 'orange', label = 'Cluster 5')
plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], s = 300, marker='x', c = 'black', label = 'Centroids')
plt.title('Klasterisasi')
plt.xlabel('Tembakau')
plt.ylabel('Tingkat Adiposa')
plt.legend()
plt.show()